In [1]:
from pynq import Overlay
from pynq import MMIO
from pynq import allocate
import pynq.lib.dma
import numpy as np
import cv2
%matplotlib inline
import matplotlib . pyplot as plt
import time

In [2]:
project_dir = "/root/jupyter_notebooks/projects/matvec_new/"
base = Overlay(project_dir + "matvec_old.bit")

In [3]:
registers = base.matvec_0.register_map
registers

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  mat = Register(mat=write-only),
  vec = Register(vec=write-only),
  out_r = Register(out_r=write-only),
  vec_len = Register(vec_len=write-only),
  out_len = Register(out_len=write-only)
}

In [4]:
vec_len = 1024 * 4
out_len = 1024 * 4
mat_shape = (out_len, vec_len)
vec_shape = (vec_len,)
out_shape = (out_len,)
mat = allocate(mat_shape, dtype=np.int32, cacheable=False)
vec = allocate(vec_shape, dtype=np.int32, cacheable=False)
out = allocate(out_shape, dtype=np.int32, cacheable=False)

In [5]:
mat[:] = np.arange(out_len * vec_len, dtype=np.int32).reshape(mat_shape)
vec[:] = np.arange(vec_len, dtype=np.int32)
out[:] = np.zeros(shape=out_shape, dtype=np.int32)

In [6]:
registers.mat.mat = mat.device_address
registers.vec.vec = vec.device_address
registers.out_r.out_r = out.device_address
registers.vec_len.vec_len = vec_len
registers.out_len.out_len = out_len

In [7]:
start = time.perf_counter()
registers.CTRL.AP_START = 1
while True:
    if registers.CTRL.AP_IDLE:
        break
end = time.perf_counter()

In [8]:
time_diff = end - start
time_diff

0.14202678999572527

In [9]:
_mat = np.zeros(shape=mat_shape, dtype=np.int32)
_vec = np.zeros(shape=vec_shape, dtype=np.int32)
_mat[:] = mat
_vec[:] = vec
start = time.perf_counter()
expected = np.matmul(_mat, _vec)
end = time.perf_counter()
time_diff = end - start
time_diff

0.07998695099377073

In [10]:
diff = np.add.reduce(expected - out)
diff

PynqBuffer(0)